# Analyzing the logs

In [1]:
# Run from the top of the repository
%cd ..

/Users/theodlz/Desktop/boom-paper


In [2]:
import pandas as pd

boom_consumer_log_fpath = "logs/boom/consumer.log"
boom_scheduler_log_fpath = "logs/boom/scheduler.log"

# To calculate BOOM wall time, take first timestamp from the consumer log as
# the start and the last timestamp of the scheduler as the end
with open(boom_consumer_log_fpath) as f:
    line = f.readline()
    t1_b = pd.to_datetime(
        line.split()[2].replace("\x1b[2m", "").replace("\x1b[0m", "")
    )
with open(boom_scheduler_log_fpath) as f:
    lines = f.readlines()
    line = lines[-3]
    t2_b = pd.to_datetime(
        line.split()[2].replace("\x1b[2m", "").replace("\x1b[0m", "")
    )

boom_wall_time = t2_b - t1_b
print(f"BOOM wall time: {boom_wall_time.total_seconds() / 60:.2f} minutes")


# To calculate Kowalski wall time, just use the supervisord logs
with open("logs/kowalski/supervisord.log") as f:
    lines = f.readlines()
    for line in lines:
        if "dask-cluster entered RUNNING state" in line:
            t1_k = pd.to_datetime("T".join(line.split()[:2]))
            break
    for line in lines:
        if "received SIGTERM indicating exit request" in line:
            t2_k = pd.to_datetime("T".join(line.split()[:2]))
            break

# print the total time of each, in minutes
kowalski_wall_time = t2_k - t1_k
print(f"Kowalski wall time: {kowalski_wall_time.total_seconds() / 60:.2f} minutes")

boom_throughput_factor = kowalski_wall_time / boom_wall_time
print(f"BOOM throughput factor: {boom_throughput_factor}")

IndexError: list index out of range

In [ ]:
# TODO: Put actual results here
kafka_ingest_strate_mbps = "XXXXX"
kafka_ingest_factor = "XXXXX"

template = r"""% This file was automatically generated; edits will be overwritten!
\newcommand{\boomthroughputfactor}{BOOM_THROUGHPUT_FACTOR}
\newcommand{\kakfaingestratembps}{KAFKA_INGEST_STRATE_MBPS}
\newcommand{\kakfaingestfactor}{KAFKA_INGEST_FACTOR}
"""

# This might be better with an f-string, but this is TeX and who wants to deal
# with escaping all the braces?
template = (
    template.replace(
        "BOOM_THROUGHPUT_FACTOR", f"{round(boom_throughput_factor):1d}"
    )
    .replace("KAFKA_INGEST_STRATE_MBPS", kafka_ingest_strate_mbps)
    .replace("KAFKA_INGEST_FACTOR", kafka_ingest_factor)
)

with open("paper/results.tex", "w") as f:
    f.write(template)